In [1]:
# Install and load the necessary libraries
library(readr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)

zip_file_path <- "census+income.zip"

extracted_dir <- "data/"

# Unzip the file
unzip(zip_file_path, exdir = extracted_dir)

# Read the adult.data file into a dataframe
adult_data_path <- file.path(extracted_dir, "adult.data")
adult_df <- read.table(adult_data_path, header = FALSE, sep = ",", quote = "\"")

# Add appropriate column names
colnames(adult_df) <- c('age','working_class','financial_weight','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hrs_per_week','native_country','income')

# Display the first few rows of the dataframe
head(adult_df)

# Create test and train splits
adult_split = initial_split(adult_df, prop = 0.75, strata = income)
adult_training = training(adult_split)
adult_testing = testing(adult_split)

# Create a table summarizing the number of observations for each class in income, sex, race
summary_table <- adult_training |>
  group_by(income, sex, race) |>
  summarize(count = n())

# Display the summary table
print("Summary Table:")
print(summary_table)

# Produce means for age, education_num, hours_per_week (vars we will likely use)
means_table <- adult_training |>
    group_by(income, sex, race) |>
    summarize(
        mean_age = mean(age),
        mean_education_num = mean(education_num),
        mean_hours_per_week = mean(hrs_per_week)
      )

print("Means Table:")
print(means_table)

# Find the number of rows with missing values
missing_rows_training <- adult_training %>%
  summarise_all(~sum(is.na(.)))

# Display the number of rows with missing values
print("Number of Rows with Missing Values:")
print(sum(missing_rows_training))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ purrr     1.0.2
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::fil

,age,working_class,financial_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hrs_per_week,native_country,income
,<int>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>
1,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
2,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
3,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
4,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
5,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
6,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


`summarise()` has grouped output by 'income', 'sex'. You can override using the
`.groups` argument.


[1] "Summary Table:"
# A tibble: 20 × 4
# Groups:   income, sex [4]
   income   sex       race                  count
   <chr>    <chr>     <chr>                 <int>
 1 " <=50K" " Female" " Amer-Indian-Eskimo"    82
 2 " <=50K" " Female" " Asian-Pac-Islander"   222
 3 " <=50K" " Female" " Black"               1078
 4 " <=50K" " Female" " Other"                 71
 5 " <=50K" " Female" " White"               5712
 6 " <=50K" " Male"   " Amer-Indian-Eskimo"   130
 7 " <=50K" " Male"   " Asian-Pac-Islander"   341
 8 " <=50K" " Male"   " Black"                959
 9 " <=50K" " Male"   " Other"                112
10 " <=50K" " Male"   " White"               9833
11 " >50K"  " Female" " Amer-Indian-Eskimo"    10
12 " >50K"  " Female" " Asian-Pac-Islander"    35
13 " >50K"  " Female" " Black"                 67
14 " >50K"  " Female" " Other"                  6
15 " >50K"  " Female" " White"                755
16 " >50K"  " Male"   " Amer-Indian-Eskimo"    16
17 " >50K"  " Male"   " Asian-Pa

`summarise()` has grouped output by 'income', 'sex'. You can override using the
`.groups` argument.


[1] "Means Table:"
# A tibble: 20 × 6
# Groups:   income, sex [4]
   income   sex       race       mean_age mean_education_num mean_hours_per_week
   <chr>    <chr>     <chr>         <dbl>              <dbl>               <dbl>
 1 " <=50K" " Female" " Amer-In…     37.2               9.21                35.5
 2 " <=50K" " Female" " Asian-P…     33.7              10.3                 37.4
 3 " <=50K" " Female" " Black"       37.9               9.44                36.7
 4 " <=50K" " Female" " Other"       29.6               8.89                35.1
 5 " <=50K" " Female" " White"       36.2               9.89                35.5
 6 " <=50K" " Male"   " Amer-In…     36.4               8.88                41.6
 7 " <=50K" " Male"   " Asian-P…     37.6              10.6                 39.2
 8 " <=50K" " Male"   " Black"       36.2               9.06                38.5
 9 " <=50K" " Male"   " Other"       33.7               8.41                41.2
10 " <=50K" " Male"   " White"       37.2  